In [2]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from torch.utils.data import TensorDataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


In [26]:
## Sklearn Stuff
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Dataset preparation

In [5]:
train_instances = pd.read_csv("../data_forecasting/Train.csv")
train_instances.tail()

,year,month,region,district,site_code,product_code,stock_initial,stock_received,stock_distributed,stock_adjustment,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered
35748,2016,7,LOH-DJIBOUA,GUITRY,C2055,AS27133,0,50,25,0,25,8,0,25.0
35749,2016,7,LOH-DJIBOUA,GUITRY,C2055,AS27138,150,0,0,0,150,0,0,0.0
35750,2016,7,LOH-DJIBOUA,GUITRY,C2055,AS27132,1,10,4,0,7,13,0,0.0
35751,2016,7,LOH-DJIBOUA,GUITRY,C2055,AS27134,8,0,8,0,0,10,0,0.0
35752,2016,7,LOH-DJIBOUA,GUITRY,C2055,AS17005,48,0,2,0,46,1,0,0.0


In [6]:
train_instances['timestamp'] = train_instances['year'].astype('str') + "-" + train_instances['month'].astype('str')
train_instances.drop(columns={'region', 
                      'district', 
                      'stock_initial', 
                      'stock_received', 
                      'stock_adjustment',
                      'stock_end',
                      'average_monthly_consumption',
                      'stock_stockout_days',
                      'stock_ordered'},
                      inplace=True)

In [7]:
train_instances['timestamp']=pd.to_datetime(train_instances['timestamp'])
train_instances.index= train_instances.timestamp
train_instances.drop(columns={'timestamp'},inplace=True)

## Sort Dataframe by Date - Causality in Preds
train_instances.sort_values(by='timestamp', inplace=True)
train_instances.tail()

,year,month,site_code,product_code,stock_distributed
timestamp,,,,,
2019-06-01,2019,6,C1007,AS27138,23
2019-06-01,2019,6,C1007,AS27137,11
2019-06-01,2019,6,C1007,AS27139,0
2019-06-01,2019,6,C5001,AS27137,2
2019-06-01,2019,6,C2052,AS27134,0


In [9]:
site_encoder = LabelEncoder()
product_encoder = LabelEncoder()
#train_instances.drop(columns=['timestamp'], inplace=True)
train_instances['site_code_encoded'] = site_encoder.fit_transform(train_instances['site_code'])
train_instances['product_code_encoded'] = product_encoder.fit_transform(train_instances['product_code'])
train_instances.tail()

,year,month,site_code,product_code,stock_distributed,site_code_encoded,product_code_encoded
timestamp,,,,,,,
2019-06-01,2019,6,C1007,AS27138,23,1,7
2019-06-01,2019,6,C1007,AS27137,11,1,6
2019-06-01,2019,6,C1007,AS27139,0,1,8
2019-06-01,2019,6,C5001,AS27137,2,140,6
2019-06-01,2019,6,C2052,AS27134,0,81,5


In [10]:
def onehot_encode_pd(df, col_name):
    dummies = pd.get_dummies(df[col_name], prefix=col_name)
    return pd.concat([df, dummies], axis=1).drop(columns=[col_name])

In [13]:
final_dataset=train_instances
#final_dataset=train_instances[['site_code_encoded', 'product_code_encoded', 'stock_distributed']]
final_dataset.tail()

,year,month,site_code,product_code,stock_distributed,site_code_encoded,product_code_encoded
timestamp,,,,,,,
2019-06-01,2019,6,C1007,AS27138,23,1,7
2019-06-01,2019,6,C1007,AS27137,11,1,6
2019-06-01,2019,6,C1007,AS27139,0,1,8
2019-06-01,2019,6,C5001,AS27137,2,140,6
2019-06-01,2019,6,C2052,AS27134,0,81,5


In [14]:
final_df = final_dataset[['year','month','site_code_encoded','product_code_encoded','stock_distributed']].reset_index(drop=True)
final_df

,year,month,site_code_encoded,product_code_encoded,stock_distributed
0,2016,1,10,6,2
1,2016,1,32,2,0
2,2016,1,32,3,0
3,2016,1,32,6,0
4,2016,1,32,7,0
...,...,...,...,...,...
35748,2019,6,1,7,23
35749,2019,6,1,6,11
35750,2019,6,1,8,0
35751,2019,6,140,6,2


In [15]:
## Train/Test Split
train_size = 0.8
train = final_dataset[:int(train_size*(len(final_dataset)))]
valid = final_dataset[int(train_size*(len(final_dataset))):]

In [17]:
## Min/Max Scaler
scaler = MinMaxScaler()
X_train_arr = scaler.fit_transform(train[['year','month','site_code_encoded','product_code_encoded']])
X_val_arr = scaler.transform(valid[['year','month','site_code_encoded','product_code_encoded']])

y_train_arr = scaler.fit_transform(train[['stock_distributed']])
y_val_arr = scaler.transform(valid[['stock_distributed']])

In [20]:
## Pytorch Data loaders
batch_size = 64

train_features = torch.Tensor(X_train_arr)
train_targets = torch.Tensor(y_train_arr)
val_features = torch.Tensor(X_val_arr)
val_targets = torch.Tensor(y_val_arr)

train = TensorDataset(train_features, train_targets)
val = TensorDataset(val_features, val_targets)

train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)

## PyTorch Modeling

In [21]:
class LSTMPredictor(nn.Module):
    def __init__(self, n_features, n_hidden, seq_len, n_layers=2):
        super(LSTMPredictor, self).__init__()
        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers
        self.lstm = nn.LSTM(
          input_size=n_features,
          hidden_size=n_hidden,
          num_layers=n_layers,
          dropout=0.5
        )
        self.linear = nn.Linear(in_features=n_hidden, out_features=1)
        self.reset_hidden_state()
        
    def reset_hidden_state(self):
        self.hidden = (
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden),
            torch.zeros(self.n_layers, self.seq_len, self.n_hidden)
        )
    def forward(self, sequences):
        lstm_out, self.hidden = self.lstm(
          sequences.view(len(sequences), self.seq_len, -1),
          self.hidden
        )
        last_time_step = \
          lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
        y_pred = self.linear(last_time_step)
        return y_pred

In [22]:
input_dim = X_train_arr.shape[1]
output_dim = 1
hidden_dim = 64
layer_dim = 3
batch_size = 64
dropout = 0.2
n_epochs = 50
learning_rate = 1e-3
weight_decay = 1e-6

In [23]:
model_params = {
    'n_features' : input_dim,
    'n_hidden': hidden_dim,
    'seq_len': output_dim,
    'n_layers': layer_dim
}

In [24]:
model = LSTMPredictor(**model_params)

loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [25]:
x, y = next(iter(train_loader))

In [38]:
for epoch in range(n_epochs):
    print(f'Epoch {epoch}')
    for step, (x,y) in enumerate(train_loader):
        model.train()
        model.reset_hidden_state()
        output = model(x)
        loss = loss_fn(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step%100 == 0:
            for step_valid, (x_valid,y_valid) in enumerate(val_loader):
                model.eval()
                output_valid = model.forward(x_valid)
                mse = mean_squared_error(y_valid.detach().numpy(), output_valid.detach().numpy())
            print(f"Iteration: {step}; Loss: {loss.item()}; MSE: {mse}")

Epoch 0
Iteration: 0; Loss: 0.2551129162311554; MSE: 0.25299593806266785
Iteration: 100; Loss: 0.0016144950641319156; MSE: 0.000311427895212546
Iteration: 200; Loss: 0.0002509684709366411; MSE: 0.0003115025465376675
Iteration: 300; Loss: 0.0013547282433137298; MSE: 0.0003081829927396029
Iteration: 400; Loss: 0.000363128783646971; MSE: 0.0003210721188224852
Epoch 1
Iteration: 0; Loss: 0.0009607175015844405; MSE: 0.0003247470303904265
Iteration: 100; Loss: 0.0015924271428957582; MSE: 0.00030247337417677045
Iteration: 200; Loss: 9.686400153441355e-05; MSE: 0.0003313015913590789
Iteration: 300; Loss: 0.0013551040319725871; MSE: 0.0003121643094345927
Iteration: 400; Loss: 0.00028418778674677014; MSE: 0.0003063111216761172
Epoch 2
Iteration: 0; Loss: 0.0009579588659107685; MSE: 0.0003226913686376065
Iteration: 100; Loss: 0.001659779460169375; MSE: 0.00030199188040569425
Iteration: 200; Loss: 5.207574213272892e-05; MSE: 0.0003249035798944533
Iteration: 300; Loss: 0.0013107256963849068; MSE: 0

KeyboardInterrupt: 

## Sources
- https://colab.research.google.com/drive/1enI68fTdPI2w5KKv6jyL0Lcq9Zg3BbLx?usp=sharing#scrollTo=A5guw4joPA8q
- https://curiousily.com/posts/time-series-forecasting-with-lstm-for-daily-coronavirus-cases/